In [ ]:
!pip install hvplot holoviews datashader geoviews datashader tarfile tqdm xarray radolan_to_netcdf numpy==2.0 cartopy jupyter_bokeh


In [ ]:
import tarfile
import hvplot.pandas
import holoviews
import datashader
import geoviews
import holoviews as hv
hv.extension('bokeh', 'matplotlib')  # Oder je nachdem, welches Backend du nutzt
# import datashader
# import geoviews
# import cartopy
import tqdm
import xarray as xr
import hvplot.xarray
import numpy
import radolan_to_netcdf as rtn
import rioxarray as rxr
import os

ModuleNotFoundError: No module named 'osgeo'

# Download RADKLIM-YW data for May 2016

We want to look at the data for the flooding in the city of Braunsbach on 29th of May 2016.

**Note that RADKLIM data is corrected based on long-term statistics. Hence, these correction are not adjusted for individual events, like the one shown here. Like all radar rainfall estimate, and all rainfall estimates in general, the absolute values have to be treated with caution. The data is shown here only to highlight how easy it now is to access and explore RADOLAN and RADKLIM data.**

In [ ]:
# !curl -O https://opendata.dwd.de/climate_environment/CDC/grids_germany/5_minutes/radolan/reproc/2017_002/bin/2016/YW2017.002_201605.tar

# Read data from nested tar file

Data is provided as monthly tar files, which contains dailt tar.gz files, which contain the 5-minute binary files. To avoid extracting everything first we use the nested loop-construct below and extract only the data we want on the fly.

In [2]:
# Pfad zur Quelldatei
fn_tar = './data/YW2017.002_202107_direct.tar'

# Pfad zur Zieldatei
fn_netcdf = './output/radklim-yw_event_20210715_16.nc'

# Start- und Endindizes für die Dateien im Hauptarchiv
file_start_index = 14  # Startfile: Index 14
file_end_index = 16    # Endfile: Index 15 (exklusive)

# Zeitbereichs-Indizes innerhalb der inneren Dateien
start_hour_index = 264  # 22:00 Uhr (Index für 14. File)
end_hour_index = 49     # 04:00 Uhr (Index für 15. File, exklusiv)

In [3]:
# Erstelle ein leeres NetCDF-File zur Speicherung der zusammengefügten Daten
rtn.create_empty_netcdf(fn=fn_netcdf, product_name='YW')

# Öffne das Haupt-tar-Archiv, welches mehrere Dateien mit Zeitseriendaten enthält
with tarfile.open(fn_tar, 'r') as tar:
    fn_list = tar.getnames()  # Erhalte die Liste aller Dateien im Archiv
    fn_list.sort()  # Sortiere die Dateien im Archiv für eine geordnete Extraktion

    # Extrahiere die erste Datei des gewünschten Bereichs im Hauptarchiv
    f_start = tar.extractfile(fn_list[file_start_index])
    with tarfile.open(fileobj=f_start) as tar_inner_start:
        fn_list_inner_start = tar_inner_start.getnames()  # Erhalte die Liste aller Zeitschritte
        fn_list_inner_start.sort()  # Sortiere die Zeitschritte innerhalb der Datei

        # Schleife über den definierten Bereich der Zeitschritte in der ersten Datei
        for fn_inner in tqdm.tqdm(fn_list_inner_start[start_hour_index:]):
            # Lese die Zeitschritt-Daten und zugehörige Metadaten ein
            data, metadata = rtn.read_in_one_bin_file(tar_inner_start.extractfile(fn_inner))
            # Füge die gelesenen Daten dem NetCDF-File hinzu
            rtn.append_to_netcdf(
                fn_netcdf, 
                data_list=[data], 
                metadata_list=[metadata],
            )

    # Extrahiere die zweite Datei des gewünschten Bereichs im Hauptarchiv
    f_end = tar.extractfile(fn_list[file_start_index + 1])  # Nächste Datei im Hauptarchiv
    with tarfile.open(fileobj=f_end) as tar_inner_end:
        fn_list_inner_end = tar_inner_end.getnames()  # Erhalte die Liste aller Zeitschritte
        fn_list_inner_end.sort()  # Sortiere die Zeitschritte innerhalb der Datei
        
        # Schleife über den definierten Bereich der Zeitschritte in der zweiten Datei
        for fn_inner in tqdm.tqdm(fn_list_inner_end[:end_hour_index]):
            # Lese die Zeitschritt-Daten und zugehörige Metadaten ein
            data, metadata = rtn.read_in_one_bin_file(tar_inner_end.extractfile(fn_inner))
            # Füge die gelesenen Daten dem NetCDF-File hinzu
            rtn.append_to_netcdf(
                fn_netcdf, 
                data_list=[data], 
                metadata_list=[metadata],
            )

100%|██████████| 48/48 [01:12<00:00,  1.51s/it]


In [5]:
ds = xr.open_dataset(fn_netcdf)
ds

<xarray.Dataset> Size: 1GB
Dimensions:          (x: 900, y: 1100, time: 72)
Coordinates:
  * x                (x) float64 7kB -443.5 -442.5 -441.5 ... 453.5 454.5 455.5
  * y                (y) float64 9kB -4.759e+03 -4.758e+03 ... -3.66e+03
    latitudes        (y, x) float64 8MB ...
    longitudes       (y, x) float64 8MB ...
  * time             (time) datetime64[ns] 576B 2021-07-15T22:00:00 ... 2021-...
Data variables:
    rainfall_amount  (time, y, x) float64 570MB ...
    maxrange         (time) int16 144B ...
    radarlocations   (time) <U59 17kB ...
    secondary        (time, y, x) float32 285MB ...
    nodatamask       (time, y, x) float32 285MB ...
    cluttermask      (time, y, x) float32 285MB ...
    radolan_grid     float64 8B ...
Attributes:
    title:        RADOLAN YW rainfall data
    producttype:  YW
    institution:  Deutscher Wetterdienst (DWD)
    history:      Created at 2024-10-25 13:21:50.229726
    Conventions:  CF-1.6

In [6]:
rainfall_map = ds.rainfall_amount.hvplot.quadmesh(
    x='longitudes', 
    y='latitudes',
    frame_width=500, 
    rasterize=True,  # Setze dies vorübergehend auf False, um zu testen 
    tiles='OSM', 
    project=True, 
    geo=True, 
    clim=(0.1, 10), 
    cmap='rainbow', 
    clabel='rainfall amount (mm)')

rainfall_map.opts('Image', clipping_colors={'min': 'transparent', 'NaN': 'gray'}, alpha=0.5, toolbar='above')

rainfall_map

# from IPython.display import display
# display(rainfall_map)


BokehModel(combine_events=True, render_bundle={'docs_json': {'46df2401-5c6d-44ab-bbc0-1bedb4664a1a': {'version…

# Plot rainfall accumulation over the period covered by the data

Add coordinates of the village of Braunsbach as `pandas.Dataframe`, because this currently seems to be the easiest way to add a marker to the `hvplot` map.

In [13]:
import pandas as pd
import hvplot.pandas

poi = pd.DataFrame(data={'x': [7.6845, ], 'y': [47.5881, ]})

In [14]:
print('First time stamp in data: ' + str(ds.rainfall_amount.time.values[0]))
print('Last time stamp in data : ' + str(ds.rainfall_amount.time.values[-1]))

First time stamp in data: 2021-07-15T22:00:00.000000000
Last time stamp in data : 2021-07-16T03:55:00.000000000


In [16]:
rainfall_map = ds.rainfall_amount.sum(dim='time').hvplot.quadmesh(
    x='longitudes', 
    y='latitudes',
    frame_width=500, 
    rasterize=True,
    tiles='OSM', 
    project=True, 
    geo=True, 
    clim=(0.1, 30), 
    cmap='rainbow', 
    clabel='rainfall amount (mm)')

rainfall_map.opts('Image', clipping_colors={'min': 'transparent', 'NaN': 'gray'}, alpha=0.5, toolbar='above')

rainfall_map * poi.hvplot.points(x='x', y='y', geo=True, color='black')

BokehModel(combine_events=True, render_bundle={'docs_json': {'dab52d4b-8a11-4ad8-aaf0-ca88aac623ea': {'version…

# Export timesteps to tif

In [19]:
# NetCDF-Datei laden
ds = xr.open_dataset(fn_netcdf)

# Ausgabeordner definieren
output_folder = "./output"
os.makedirs(output_folder, exist_ok=True)

# Variablenname anpassen, falls "rainfall_amount" nicht der Name der Datenvariable ist
variable_name = "rainfall_amount"

# Schleife über alle Zeitschritte und Export als GeoTIFF mit Zeitstempel im Dateinamen
for timestamp in ds.time.values:
    # Einzelnen Zeitschritt extrahieren
    ds_single_time = ds.sel(time=timestamp)
    
    # Datensatz in ein Raster-Array umwandeln und exportieren
    raster = ds_single_time[variable_name].rio.write_crs("EPSG:4326")  # Setze EPSG entsprechend an
    
    # Dateiname mit Zeitstempel als Suffix
    timestamp_str = pd.to_datetime(str(timestamp)).strftime("%Y%m%d_%H%M")
    output_path = os.path.join(output_folder, f"{variable_name}_{timestamp_str}.tif")
    
    # Speichere als GeoTIFF
    raster.rio.to_raster(output_path)
    print(f"Exported {output_path}")

print("Alle Zeitschritte erfolgreich als GeoTIFF exportiert.")

Exported ./output\rainfall_amount_20210715_2200.tif
Exported ./output\rainfall_amount_20210715_2205.tif
Exported ./output\rainfall_amount_20210715_2210.tif
Exported ./output\rainfall_amount_20210715_2215.tif
Exported ./output\rainfall_amount_20210715_2220.tif
Exported ./output\rainfall_amount_20210715_2225.tif
Exported ./output\rainfall_amount_20210715_2230.tif
Exported ./output\rainfall_amount_20210715_2235.tif
Exported ./output\rainfall_amount_20210715_2240.tif
Exported ./output\rainfall_amount_20210715_2245.tif
Exported ./output\rainfall_amount_20210715_2250.tif
Exported ./output\rainfall_amount_20210715_2255.tif
Exported ./output\rainfall_amount_20210715_2300.tif
Exported ./output\rainfall_amount_20210715_2305.tif
Exported ./output\rainfall_amount_20210715_2310.tif
Exported ./output\rainfall_amount_20210715_2315.tif
Exported ./output\rainfall_amount_20210715_2320.tif
Exported ./output\rainfall_amount_20210715_2325.tif
Exported ./output\rainfall_amount_20210715_2330.tif
Exported ./o

In [ ]:
import xarray as xr
import rioxarray as rxr
import geopandas as gpd
import os

# NetCDF-Datei laden
ds = xr.open_dataset(fn_netcdf)

# Shapefile laden (mit anderem Koordinatensystem)
shapefile_path = './path/to/shapefile.shp'  # Pfad zum Shapefile
gdf = gpd.read_file(shapefile_path)

# Ausgabeordner definieren
output_folder = "./output"
os.makedirs(output_folder, exist_ok=True)

# Variable, die steuert, ob das Zuschneiden durchgeführt werden soll
clip_data = True  # Setze auf False, um das Zuschneiden auszulassen

# Variablenname anpassen, falls "rainfall_amount" nicht der Name der Datenvariable ist
variable_name = "rainfall_amount"

# Schleife über alle Zeitschritte und Export als GeoTIFF mit Zeitstempel im Dateinamen
for timestamp in ds.time.values:
    # Einzelnen Zeitschritt extrahieren
    ds_single_time = ds.sel(time=timestamp)
    
    # Datensatz in ein Raster-Array umwandeln und exportieren
    raster = ds_single_time[variable_name].rio.write_crs("EPSG:4326")  # Setze EPSG entsprechend an
    
    if clip_data:
        # Zuschneiden der Rasterdaten mit dem Shapefile
        raster = raster.rio.clip(gdf.geometry, gdf.crs, drop=True)
    
    # Dateiname mit Zeitstempel als Suffix
    timestamp_str = pd.to_datetime(str(timestamp)).strftime("%Y%m%d_%H%M")
    output_path = os.path.join(output_folder, f"{variable_name}_{timestamp_str}.tif")
    
    # Speichere als GeoTIFF
    raster.rio.to_raster(output_path)
    print(f"Exported {output_path}")

print("Alle Zeitschritte erfolgreich als GeoTIFF exportiert.")


# WKT Definition for exported files

```
PROJCRS["Stereographic_North_Pole",BASEGEOGCRS["GCS_unnamed ellipse",DATUM["unknown",ELLIPSOID["Unknown",6370040,0,LENGTHUNIT["metre",1,ID["EPSG",9001]]]],PRIMEM["Greenwich",0,ANGLEUNIT["Degree",0.0174532925199433]]],CONVERSION["unnamed",METHOD["Polar Stereographic (variant B)",ID["EPSG",9829]],PARAMETER["Latitude of standard parallel",60,ANGLEUNIT["Degree",0.0174532925199433],ID["EPSG",8832]],PARAMETER["Longitude of origin",10,ANGLEUNIT["Degree",0.0174532925199433],ID["EPSG",8833]],PARAMETER["False easting",0,LENGTHUNIT["metre",1000],ID["EPSG",8806]],PARAMETER["False northing",0,LENGTHUNIT["metre",1000],ID["EPSG",8807]]],CS[Cartesian,2],AXIS["x",south,ORDER[1],LENGTHUNIT["metre",1000]],AXIS["y",south,ORDER[2],LENGTHUNIT["metre",1000]]]
```

In [20]:
# create Gauss Krueger zone 3 projection osr object
proj_gk3 = osr.SpatialReference()
proj_gk3.ImportFromEPSG(31467)

# transform radolan polar stereographic projection to wgs84 and then to gk3
radolan_grid_ll = wrl.georef.reproject(radolan_grid_xy,
                                       projection_source=proj_stereo,
                                       projection_target=proj_wgs)
radolan_grid_gk = wrl.georef.reproject(radolan_grid_ll,
                                       projection_source=proj_wgs,
                                       projection_target=proj_gk3)

lon_wgs0 = radolan_grid_ll[:, :, 0]
lat_wgs0 = radolan_grid_ll[:, :, 1]

x_gk3 = radolan_grid_gk[:, :, 0]
y_gk3 = radolan_grid_gk[:, :, 1]

x_rad = radolan_grid_xy[:, :, 0]
y_rad = radolan_grid_xy[:, :, 1]

print("\n------------------------------")
print("source radolan x,y-coordinates")
print(u"       {0}      {1} ".format('x [km]', 'y [km]'))
print("ll: {:10.4f} {:10.3f} ".format(x_rad[0, 0], y_rad[0, 0]))
print("lr: {:10.4f} {:10.3f} ".format(x_rad[0, -1], y_rad[0, -1]))
print("ur: {:10.4f} {:10.3f} ".format(x_rad[-1, -1], y_rad[-1, -1]))
print("ul: {:10.4f} {:10.3f} ".format(x_rad[-1, 0], y_rad[-1, 0]))
print("\n--------------------------------------")
print("transformed radolan lonlat-coordinates")
print(u"      {0}  {1} ".format('lon [degE]', 'lat [degN]'))
print("ll: {:10.4f}  {:10.4f} ".format(lon_wgs0[0, 0], lat_wgs0[0, 0]))
print("lr: {:10.4f}  {:10.4f} ".format(lon_wgs0[0, -1], lat_wgs0[0, -1]))
print("ur: {:10.4f}  {:10.4f} ".format(lon_wgs0[-1, -1], lat_wgs0[-1, -1]))
print("ul: {:10.4f}  {:10.4f} ".format(lon_wgs0[-1, 0], lat_wgs0[-1, 0]))
print("\n-----------------------------------")
print("transformed radolan gk3-coordinates")
print(u"     {0}   {1} ".format('easting [m]', 'northing [m]'))
print("ll: {:10.0f}   {:10.0f} ".format(x_gk3[0, 0], y_gk3[0, 0]))
print("lr: {:10.0f}   {:10.0f} ".format(x_gk3[0, -1], y_gk3[0, -1]))
print("ur: {:10.0f}   {:10.0f} ".format(x_gk3[-1, -1], y_gk3[-1, -1]))
print("ul: {:10.0f}   {:10.0f} ".format(x_gk3[-1, 0], y_gk3[-1, 0]))

ModuleNotFoundError: No module named 'osgeo'